In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U trl
#!pip install -q -U git+https://github.com/huggingface/accelerate.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install -U bitsandbytes

In [5]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,              # 리포지토리의 custom 코드를 실행하도록 설정
    quantization_config=quant_config,      # BitsAndBytesConfig 객체 전달
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [7]:
model

ExaoneForCausalLM(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 4096, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSdpaAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
        )
        (ln_2): ExaoneRMSNorm()
        (mlp): ExaoneGatedMLP(
          (c_fc_0): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (c_fc_1): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (c_proj): Linear4bit(in_features=14336, out_features=4096, bias=

In [8]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Exaone 모델 구조에 맞게 target_modules 목록을 수정
lora_config = LoraConfig(
    r=4,                     # LoRA 가중치 행렬의 rank (값이 작을수록 trainable parameter 수가 줄어듦)
    lora_alpha=8,            # LoRA 스케일링 팩터
    lora_dropout=0.05,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'out_proj', 'c_fc_0', 'c_fc_1', 'c_proj'],
    bias='none',             # bias 파라미터 학습 여부: 'none', 'all', 'lora_only'
    task_type="CAUSAL_LM"     # 문자열로 task_type 지정 (이전에는 TaskType.CAUSAL_LM을 사용했음)
)

# 양자화된 모델 학습 전 전처리
model = prepare_model_for_kbit_training(model)

# PEFT를 사용해 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 출력
model.print_trainable_parameters()


trainable params: 10,485,760 || all params: 7,828,934,656 || trainable%: 0.1339


In [9]:
train = pd.read_csv("/content/drive/MyDrive/train.csv", encoding = 'utf-8-sig')

In [10]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

In [11]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [14]:
from datasets import Dataset

# combined_training_data가 Pandas DataFrame인 경우
dataset_hf = Dataset.from_pandas(combined_training_data.reset_index(drop=True))


In [15]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 특수문자를 포함하지 마세요.

{context}

### 질문:
{question}

### 답변:
"""

In [16]:
def generate_prompt(data_point):
    # 추가 context가 필요한 경우 여기서 설정 (현재는 빈 문자열)
    context = ""
    question = data_point["question"]
    # prompt_template을 사용해 prompt를 생성합니다.
    prompt = prompt_template.format(context=context, question=question)
    # 모델 입력으로 사용할 프롬프트와 정답(평가용)만 반환합니다.
    return {"prompt": prompt, "answer": data_point["answer"]}

# 기존 데이터셋의 모든 컬럼을 제거하고, generate_prompt에서 필요한 컬럼만 남깁니다.
remove_column_keys = list(dataset_hf.features.keys())
dataset_cvted = dataset_hf.shuffle().map(generate_prompt, remove_columns=remove_column_keys)


Map:   0%|          | 0/23422 [00:00<?, ? examples/s]

In [17]:
dataset_cvted

Dataset({
    features: ['answer', 'prompt'],
    num_rows: 23422
})

In [18]:
print(dataset_cvted[0])

{'answer': '작업자 재교육 실시 및 사고 발생 전파와 유사사례 발생 방지를 위한 재교육 실시.', 'prompt': "\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n공사종류 대분류 '산업환경설비', 중분류 '환경시설' 공사 중 공종 대분류 '토목', 중분류 '철근콘크리트공사' 작업에서 사고객체 '가시설'(중분류: '거푸집')와 관련된 사고가 발생했습니다. 작업 프로세스는 '해체작업'이며, 사고 원인은 '유량조정조 천정 합판 거푸집 해체 작업 중 무리한 힘으로 거푸집을 해체하여 벽체와 작업공구(빠루)사이  협착 발생'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"}


In [19]:
def tokenize_function(examples):
    outputs = tokenizer(examples["prompt"], truncation=True, max_length=512)
    return outputs


In [20]:
remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

Map:   0%|          | 0/23422 [00:00<?, ? examples/s]

In [ ]:
dataset_tokenized

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 23422
})

In [30]:
# 토크나이징 완료된 데이터셋을 train/test로 분리
split_datasets = dataset_tokenized.train_test_split(test_size=0.1, seed=42)

train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]  # 보통 검증 용도로 사용

In [21]:
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [32]:
# 2. TrainingArguments에 evaluation_strategy와 eval_steps 추가
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=8,         # 각 디바이스당 배치 사이즈
    gradient_accumulation_steps=4,
    warmup_steps=1,
    max_steps=2000,
    learning_rate=2e-4,                     # 학습률
    bf16=True,                              # bf16 사용 (하드웨어 지원 확인 필요)
    output_dir="outputs",
    optim="paged_adamw_8bit",               # 8비트 AdamW 옵티마이저
    logging_steps=50,                       # 로깅 빈도
    save_total_limit=3,                     # 저장할 체크포인트의 최대 수
    evaluation_strategy="steps",            # 일정 스텝마다 평가 실행
    eval_steps=50                           # 평가 주기 (50 스텝마다 평가)
)

# 3. SFTTrainer에 eval_dataset을 추가하여 수정
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=train_args,
    data_collator=collate_fn
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/21079 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/21079 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/21079 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/2343 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/2343 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2343 [00:00<?, ? examples/s]

In [33]:
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,0.339600,0.337044


KeyboardInterrupt: 

In [34]:
# 저장 - > 이때 qLoRA를 이용해서 학습한 파라미터만 저장되서 작음
FINETUNED_MODEL = "qlora"
trainer.model.save_pretrained(FINETUNED_MODEL)

In [37]:
from peft import PeftConfig

In [38]:
#불러오기
FINETUNED_MODEL = "/content/qlora"

peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [40]:
# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path
)

The repository for LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] ㅛ
The repository for LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [41]:
# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [42]:
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [63]:
prompt = dataset_cvted[2]['prompt']
# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=64)
outputs = pipe(
    prompt,
    do_sample=False,
    #temperature=0.1,
    top_k=0,
    #top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


신규 작업자 교육 강화 프로그램 도입과 함께 정기적인 현장 감독 체계 구축하여 즉시 피드백 제공 시스템 마련하고, 모든 관공사 매설작업 전 철저한 사전안전교육 의무화합니다._위험 요소 식별 훈련도 병행해 지속적으로 개선할 것 입니다_.


In [59]:
dataset_cvted[2]

{'answer': '위험공종 숙련공 투입 및 신규 근로자 투입 금지, 위험요소 파악 후 근로자 투입 전 안전교육 실시 철저.',
 'prompt': "\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n공사종류 대분류 '토목', 중분류 '도로' 공사 중 공종 대분류 '토목', 중분류 '관공사' 작업에서 사고객체 'nan'(중분류: 'nan')와 관련된 사고가 발생했습니다. 작업 프로세스는 '매설작업'이며, 사고 원인은 '신규근로자 투입에 따른 관리감독 미흡, 위험요소 관련 근로자 안전교육 실시 미흡'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"}

In [ ]:
import pandas as pd
from datasets import Dataset
from tqdm.auto import tqdm

# 1. CSV 파일 로드 (예: test.csv)
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")

# 데이터 전처리
test_df['공사종류(대분류)'] = test_df['공사종류'].str.split(' / ').str[0]
test_df['공사종류(중분류)'] = test_df['공사종류'].str.split(' / ').str[1]
test_df['공종(대분류)'] = test_df['공종'].str.split(' > ').str[0]
test_df['공종(중분류)'] = test_df['공종'].str.split(' > ').str[1]
test_df['사고객체(대분류)'] = test_df['사고객체'].str.split(' > ').str[0]
test_df['사고객체(중분류)'] = test_df['사고객체'].str.split(' > ').str[1]

# 2. train 데이터와 동일한 방식으로 prompt 생성
def generate_prompt_from_row(row):
    prompt = (
        f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
        f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
        f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
        f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
        f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
    )
    return {"prompt": prompt}

# DataFrame의 각 행에 대해 prompt 생성 (새로운 열로 추가)
test_df = test_df.apply(generate_prompt_from_row, axis=1)
test_df = pd.DataFrame(test_df.tolist())

# 3. HuggingFace Dataset으로 변환
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# (선택사항) 토크나이즈 적용 - 보통 inference에서는 원본 prompt를 사용합니다.
# 만약 tokenizer를 이용한 전처리가 필요하다면 아래와 같이 map 적용
remove_column_keys = test_dataset.features.keys()
test_dataset_tokenized = test_dataset.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

# 4. 모델 파이프라인을 사용해 결과 생성
# 배치 단위로 추론하면서 tqdm으로 진행 상황을 모니터링합니다.
batch_size = 16
prompts = test_dataset["prompt"]
results = []

for i in tqdm(range(0, len(prompts), batch_size), desc="Generating outputs"):
    batch_prompts = prompts[i:i+batch_size]
    batch_results = pipe(
        batch_prompts,
        do_sample=False,
        top_k=0,
        repetition_penalty=1.2,
        add_special_tokens=True
    )
    results.extend(batch_results)

# 5. 결과 출력
for res in results:
    print(res)


Map:   0%|          | 0/964 [00:00<?, ? examples/s]

Generating outputs:   0%|          | 0/61 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv('./sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')